# Базы данных. Часть 2

## Базы данных и Python

Питон позволяет работать с различными СУБД, но проще всего использовать встроенную библиотеку [SQLite](https://docs.python.org/3.5/library/sqlite3.html). Важно также запомнить, что файлы баз данных имеют расширение `.db`.

In [13]:
import sqlite3

# подключаемся к базе данных
conn = sqlite3.connect('example2.db')

**Важная вещь №1**: после подключения к БД нужно обязательно инициализировать курсор, иначе вы не сможете делать к ней запросы!

In [14]:
# создаем объект "курсор", которому будем передавать запросы
c = conn.cursor()

# создаем таблицу
c.execute("CREATE TABLE IF NOT EXISTS students(name text, major text, year integer)")

# вставляем строку
c.execute("INSERT INTO students VALUES ('Петя','филология',1), ('Настя','лингвистика',4)")

# сохраняем изменения
conn.commit()

# отключаемся от БД
#conn.close()

**Важная вещь №2**: если вы что-то изменили, нужно обязательно закоммитить изменения, иначе они не сохранятся в файле БД!

Когда вы создали базу данных и ее закоммитили, вы можете открыть её через DBrowser (flashback прошлый семинар) и проверить правильно ли она у вас создалась.

**И немного о безопасности**: при создании запроса нельзя использовать конкатенацию строк и форматирование строк, как в питоне. Это сделает ваше приложение уязвимым для SQL-инъекций - особых хакерских атак, которые заключаются в подставлении в запрос нежелательных комманд - например, DROP TABLE. Поподробнее об этом можно почитать [вот здесь](https://habrahabr.ru/post/148151/).

![](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)

In [3]:
# Так нельзя!
name = 'Петя'
c.execute("SELECT * FROM students WHERE name = '%s'" % name)

# Вот как надо
x = ('Петя',)
c.execute('SELECT * FROM students WHERE name=?', x)
print(c.fetchone())

('Петя', 'филология', 1)


In [4]:
# Если результатом запроса является несколько строк, можно по ним итерировать

for row in c.execute('SELECT * FROM students ORDER BY year'):
    print(row)

('Петя', 'филология', 1)
('Настя', 'лингвистика', 4)


In [5]:
# как подставить несколько переменных в sql-запрос

x = 'Аня'
y = 'математика'
z = 3

c.execute('INSERT INTO students VALUES (?, ?, ?)', (x, y, z))
conn.commit()

#### Форматирование строк

Если нужно подставить переменные в качестве названия таблицы или колонок, то придется использовать форматирование строк.

In [6]:
params = ['vowel', 'f1', 'f2']
c.execute('CREATE TABLE vowels({}, {}, {})'.format(params[0], params[1], params[2]))

In [7]:
# как написать длинный запрос посимпатичнее
c.execute('''
INSERT INTO vowels 
VALUES 
('a', 1234.56, 4567.8), 
('u', 1111.1, 3333.3)'''
)

In [8]:
for row in c.execute('SELECT * FROM vowels'):
    print(row)

('a', 1234.56, 4567.8)
('u', 1111.1, 3333.3)


#### Функции курсора

* **fetchone()** -- возвращает следующий элемент из результата запроса (т.е. одну строку из бд). Результат -- кортеж, где элементом является значение каждой из колонок или None
* **fetchall()** -- возвращает все результаты запроса в виде списка
* **fetchmany()** -- взвращает заданное количество строк из результатов запроса

In [9]:
# извлекаем строки по одной
# обратите внимание, что после каждого вызова fetchone возвращает следующую строку!
c.execute('SELECT * FROM students ORDER BY year')
print(c.fetchone())
print(c.fetchone())
print(c.fetchone())

('Петя', 'филология', 1)
('Аня', 'математика', 3)
('Настя', 'лингвистика', 4)


In [10]:
# извлекаем две строки
c.execute('SELECT * FROM students ORDER BY year')
print(c.fetchmany(2))

[('Петя', 'филология', 1), ('Аня', 'математика', 3)]


In [11]:
# извлекаем все строки
c.execute('SELECT * FROM students ORDER BY year')
print(c.fetchall())

[('Петя', 'филология', 1), ('Аня', 'математика', 3), ('Настя', 'лингвистика', 4)]


In [26]:
# Не забудьте отключить от бд, чтобы не тратить память :)
conn.close()

# Практика #2

* Прочитайте таблицу из файла `rutul_vowels.csv` (находится в папке с семинаром) в датафрейм и посмотрите с помощью .head(), что там вообще за колонки. Или можете просто посмотреть, что это за таблица в notepad++ у себя на компе

Записать данные из файла в новую базу данных можно двумя способами:

1. модуль csv и метод `csv.DictReader()`. Создайте новую базу данных и создайте в этой базе данных новую таблицу с колонками, соответствующими колонкам в rutul_vowels. Прочитать файл и построчно записать всё в базу данных с помощью `c.executemany()`
2. В модуле pandas есть функция `df.to_sql()`. Тут нужно только создать новый connection и новый cursor. Самостоятельно делать CREATE новой таблицы с правильными названиями всех колонок - не надо! В целом этот способо в сто раз круче.

### Ответик

In [59]:
import pandas as pd
import csv
import sqlite3

In [23]:
df = pd.read_csv('rutul_vowels.csv', sep=',', encoding='utf-8')

In [67]:
df.head()

,id,word,translation,vowel,stress,syllable,left_context
0,1,χed,дикая алыча,e,yes,cvc,χ
1,2,eked,старший,e,no,cv,no
2,3,ɢina,в Кинe,i,yes,cv,ɢ
3,4,χuda,в кулаке,u,no,cv,χ
4,5,ʁuli,в окне,u,no,cv,ʁ


In [70]:
con = sqlite3.connect("rutul.db")
cur = con.cursor()

Воспользуемся вторым, более клёвым способом:

In [71]:
df.to_sql(name='rutul', con=con, if_exists='replace')

Проверим, что таблица базы данных созадалась:

(Либо можно открыть в DBrowser и поглядеть там, что всё нормально)

In [66]:
for row in cur.execute('SELECT * FROM rutul'):
    print(row)

(0, 1, 'χed', 'дикая алыча', 'e', 'yes', 'cvc', 'χ')
(1, 2, 'eked', 'старший', 'e', 'no', 'cv', 'no')
(2, 3, 'ɢina', 'в Кинe', 'i', 'yes', 'cv', 'ɢ')
(3, 4, 'χuda', 'в кулаке', 'u', 'no', 'cv', 'χ')
(4, 5, 'ʁuli', 'в окне', 'u', 'no', 'cv', 'ʁ')
(5, 6, 'ɢuje', 'в яме', 'u', 'yes', 'cv', 'ɢ')
(6, 7, 'qaka', 'отдай назад', 'a', 'yes', 'cv', 'q')
(7, 8, 'qiq’a', 'возвращайся', 'i', 'yes', 'cv', 'q')
(8, 9, 'χɨd', 'липа', 'ɨ', 'yes', 'cvc', 'χ')
(9, 10, 'ʁɨˁbar', 'лягушки', 'ɨˁ', 'no', 'cv', 'ʁ')
(10, 11, 'χaˁrad', 'масло', 'aˁ', 'no', 'cv', 'χ')
(11, 12, 'itɨd', 'медовый', 'i', 'yes', 'cv', 'no')
(12, 13, 'uˁbra', 'мерка для муки', 'uˁ', 'no', 'cv', 'no')


Первый способ записи csv в базу данных работает как-то так (можете потом тоже поробовать):

```
con = sqlite3.connect("rutul2.db")
cur = con.cursor()
cur.execute("CREATE TABLE t (col1, col2);")  # тут надо аккуратно переписать названия всех колонок

with open('data.csv','rb') as fin:
    dr = csv.DictReader(fin)  # запятая является разделителем по умолчанию
    to_db = [(i['col1'], i['col2']) for i in dr]  # снова переписать имена колонок

cur.executemany("INSERT INTO t (col1, col2) VALUES (?, ?);", to_db)  # и снова переписать имена колонок ааа ужасно
con.commit()
con.close()
```

### Вторая часть этого задания на составление запросов

Теперь можно написать какие-нибудь запросы к созданной базе данных.

* Распечатайте все слова, в которых целевая гласная находится в безударной позиции
* Распечатайте все слова, в которых целевая гласная в безударной позиции и нет левого контекста

In [ ]:
# Тут нужно писать запросы к получившейся базе



In [72]:
# Не забыть!
con.close()